## Dividend based investing

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import math
from scipy import stats

## Loading the Tickers

In [4]:
tickers = pd.read_csv('top_50_indian_stocks.csv')
tickers.head(10)

,Ticker,Company Name
0,RELIANCE.NS,Reliance Industries
1,TCS.NS,Tata Consultancy Services
2,HDFCBANK.NS,HDFC Bank
3,INFY.NS,Infosys
4,ICICIBANK.NS,ICICI Bank
5,HINDUNILVR.NS,Hindustan Unilever
6,SBIN.NS,State Bank of India
7,BAJFINANCE.NS,Bajaj Finance
8,BHARTIARTL.NS,Bharti Airtel
9,ITC.NS,ITC


## Weighted Scoring Model for finding the final score

In [5]:
def create_dividend_df(tickers):
    columns = [
        "Ticker", 
        "Dividend Yield(%)", 
        "Dividend Rate",
        "Payout Ratio(%)", 
        "5 Year Avg Dividend Yield(%)", 
        "Earning Growth(%)"
        
    ]
    
    dividend_df = pd.DataFrame(columns=columns)
    
    for stock in tickers:
        ticker = yf.Ticker(stock)
        info = ticker.info 
        
        dividend_yield = info.get("dividendYield", np.nan) * 100 if info.get("dividendYield") else np.nan
        dividend_rate = info.get("dividendRate", np.nan) 
        payout_ratio = info.get("payoutRatio", np.nan) * 100 if info.get("payoutRatio") else np.nan
        five_year_avg_dividend_yield = info.get("fiveYearAvgDividendYield", np.nan) * 100 if info.get("fiveYearAvgDividendYield") else np.nan
        earning_growth = info.get("earningsGrowth", np.nan) * 100 if info.get("earningsGrowth") else np.nan
        
        dividend_df.loc[len(dividend_df)] = [stock, dividend_yield, dividend_rate, payout_ratio, five_year_avg_dividend_yield, earning_growth]
        
    ## we have got our df with dividend information
    
    numeric_cols = [
          "Dividend Yield(%)", 
        "Dividend Rate",
        "Payout Ratio(%)", 
        "5 Year Avg Dividend Yield(%)", 
        "Earning Growth(%)"
    ]
    

    for col in numeric_cols:
        if col == "Payout Ratio":
            dividend_df[col+ " Normalised"] = 1 - ((dividend_df[col] - dividend_df[col].min()) / (dividend_df[col].max() - dividend_df[col].min()))
        else:
            dividend_df[col+ " Normalised"] = (dividend_df[col] - dividend_df[col].min()) / (dividend_df[col].max() - dividend_df[col].min())
        
    return dividend_df

In [6]:
tickers_list = tickers["Ticker"].values.tolist()
dividend_df = create_dividend_df(tickers_list)
dividend_df

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised
0,RELIANCE.NS,0.400000,5.00,9.960000,44.0,-4.8,0.028169,0.036145,0.058941,0.042082,0.109364
1,TCS.NS,0.970000,40.00,42.810002,128.0,6.2,0.075394,0.317269,0.277722,0.135105,0.124535
2,HDFCBANK.NS,1.190000,19.50,21.520000,67.0,39.0,0.093621,0.152610,0.135931,0.067553,0.169770
3,INFY.NS,2.130000,41.00,59.240000,212.0,4.2,0.171500,0.325301,0.387146,0.228128,0.121776
4,ICICIBANK.NS,0.830000,10.00,15.120000,99.0,18.2,0.063795,0.076305,0.093307,0.102990,0.141084
5,HINDUNILVR.NS,1.840000,43.00,96.089995,145.0,-2.4,0.147473,0.341365,0.632567,0.153931,0.112674
6,SBIN.NS,1.770000,13.70,17.090000,139.0,22.9,0.141674,0.106024,0.106427,0.147287,0.147566
7,BAJFINANCE.NS,0.470000,36.00,14.479999,24.0,10.4,0.033969,0.285141,0.089044,0.019934,0.130327
8,BHARTIARTL.NS,0.490000,8.00,38.680000,80.0,159.7,0.035626,0.060241,0.250216,0.081949,0.336229
9,ITC.NS,3.350000,15.00,83.739996,405.0,1.8,0.272577,0.116466,0.550316,0.441860,0.118466


In [7]:
weights = {
        "Dividend Yield(%) Normalised": 0.3,
        "Dividend Rate Normalised": 0.2,
        "Payout Ratio(%) Normalised": 0.2,
        "5 Year Avg Dividend Yield(%) Normalised": 0.2,
        "Earning Growth(%) Normalised": 0.1
    }

dividend_df["Dividend Score"] = dividend_df[[col for col in weights.keys()]].mul(list(weights.values())).sum(axis = 1)
dividend_df

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised,Dividend Score
0,RELIANCE.NS,0.400000,5.00,9.960000,44.0,-4.8,0.028169,0.036145,0.058941,0.042082,0.109364,0.046821
1,TCS.NS,0.970000,40.00,42.810002,128.0,6.2,0.075394,0.317269,0.277722,0.135105,0.124535,0.181091
2,HDFCBANK.NS,1.190000,19.50,21.520000,67.0,39.0,0.093621,0.152610,0.135931,0.067553,0.169770,0.116282
3,INFY.NS,2.130000,41.00,59.240000,212.0,4.2,0.171500,0.325301,0.387146,0.228128,0.121776,0.251743
4,ICICIBANK.NS,0.830000,10.00,15.120000,99.0,18.2,0.063795,0.076305,0.093307,0.102990,0.141084,0.087767
5,HINDUNILVR.NS,1.840000,43.00,96.089995,145.0,-2.4,0.147473,0.341365,0.632567,0.153931,0.112674,0.281082
6,SBIN.NS,1.770000,13.70,17.090000,139.0,22.9,0.141674,0.106024,0.106427,0.147287,0.147566,0.129206
7,BAJFINANCE.NS,0.470000,36.00,14.479999,24.0,10.4,0.033969,0.285141,0.089044,0.019934,0.130327,0.102047
8,BHARTIARTL.NS,0.490000,8.00,38.680000,80.0,159.7,0.035626,0.060241,0.250216,0.081949,0.336229,0.122792
9,ITC.NS,3.350000,15.00,83.739996,405.0,1.8,0.272577,0.116466,0.550316,0.441860,0.118466,0.315348


## Sorting the stocks according to Dividend Score

## Top 10 stocks based up the dividend score to invest into

In [8]:
dividend_df = dividend_df.sort_values(by = "Dividend Score", ascending = False)
dividend_df.head(10)

,Ticker,Dividend Yield(%),Dividend Rate,Payout Ratio(%),5 Year Avg Dividend Yield(%),Earning Growth(%),Dividend Yield(%) Normalised,Dividend Rate Normalised,Payout Ratio(%) Normalised,5 Year Avg Dividend Yield(%) Normalised,Earning Growth(%) Normalised,Dividend Score
45,IOC.NS,12.130000,15.00,90.669996,724.0,NaN,1.000000,0.116466,0.596470,0.795127,NaN,0.601613
30,COALINDIA.NS,10.770001,41.25,43.560000,909.0,-7.4,0.887324,0.327309,0.282717,1.000000,0.105779,0.598780
31,BPCL.NS,8.210000,23.00,68.360000,569.0,-72.2,0.675228,0.180723,0.447885,0.623477,0.016412,0.454627
42,HEROMOTOCO.NS,2.500000,115.00,53.990000,323.0,47.1,0.202154,0.919679,0.352181,0.351052,0.180941,0.403323
20,ONGC.NS,7.110000,18.25,37.500000,514.0,-25.2,0.584093,0.142570,0.242358,0.562569,0.081230,0.372850
22,POWERGRID.NS,5.220000,15.75,67.090000,512.0,0.5,0.427506,0.122490,0.439427,0.560354,0.116674,0.364374
28,TECHM.NS,2.550000,43.00,108.430000,222.0,153.1,0.206297,0.341365,0.714752,0.239203,0.327127,0.353666
41,TATASTEEL.NS,2.770000,3.60,151.260000,293.0,NaN,0.224524,0.024900,1.000000,0.317829,NaN,0.335903
13,HCLTECH.NS,2.410000,48.00,85.850000,285.0,9.8,0.194698,0.381526,0.564369,0.308970,0.129499,0.322332
9,ITC.NS,3.350000,15.00,83.739996,405.0,1.8,0.272577,0.116466,0.550316,0.441860,0.118466,0.315348
